# <font color='#eb3483'> Structured Query Language (SQL) </font> 
SQL is a programming language that let's us communicate with relational databases. There are a bunch of different versions of SQL out there(MySQL, SQLite, PostgresSQL ...etc.), but all of them use more or less the same syntax (nothing that a quick google search can't help sort out). In this module we're going to use SQLite3, a nice light-weight version of SQL that let's us focus on running queries without a lot of overhead. You can run SQL from the command line, but we'll be doing it through jupyter notebook.

## <font color='#eb3483'> What is A Database? </font>

A database is simply an organized `collection of data`. Data scientists and companies have several common things they would like to do with their data. That is **CREATE** new data, **READ** existing data fast, **UPDATE** data as it changes, and **DELETE** data. These are so common that they have the acronym **CRUD**.

There are many types of databases to solve this problem. Some examples are document, graph, noSQL, and key-value. However, the most common methodology for storing data is a *relational database*.

### <font color='#eb3483'> Tables </font>

A table is a tabular collection of related data, like and excel spread sheet. The columns are the attributes and the rows are data points. Bellow are several tables about pets and their owners.


| id | name  |
|----|-------|
|  1 | Bob   |
|  2 | Alice |
|  3 | Craig |
|  4 | Wendy |

| id | name  | owner_id | animal_id | sex  | birth      | 
|----|-------|----------|-----------|------|------------| 
|  1 | Fido  |        1 |         1 | M    | 2017-06-03 | 
|  3 | Coco  |        2 |         1 | F    | 2016-11-02 |  
|  4 | Molly |        2 |         1 | F    | 2012-03-27 |   
|  5 | Lulu  |        2 |         3 | F    | 2014-07-19 |   
|  6 | Jack  |        3 |         4 | M    | 2016-09-12 |
|  7 | Darcy |        4 |         2 | M    | 2015-02-10 |
|  8 | Bella |        4 |         1 | F    | 2013-04-11 |

| id | animal |
|----|--------|
|  1 | cat    |
|  3 | fish   |
|  4 | bird   |

## <font color='#eb3483'> SQL </font>

SQL stands for *Structured Query Language*. It is a universal syntax/language that can be used to access data from many different types of relational databases. We'll be using the sqlite3 package. Let's load it in and check out the help docs!

In [1]:
import sqlite3
from sqlite3 import Error #We're going to use this for some fancy error handling

In [2]:
help(sqlite3)

Help on package sqlite3:

NAME
    sqlite3

MODULE REFERENCE
    https://docs.python.org/3.10/library/sqlite3.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    The sqlite3 extension module provides a DB-API 2.0 (PEP 249) compilant
    interface to the SQLite library, and requires SQLite 3.7.15 or newer.
    
    To use the module, start by creating a database Connection object:
    
        import sqlite3
        cx = sqlite3.connect("test.db")  # test.db will be created or opened
    
    The special path name ":memory:" can be provided to connect to a transient
    in-memory database:
    
        cx = sqlite3.connect(":memory:")  # connect to a database in RAM
    
    Once a connection has 

### <font color='#eb3483'> Connecting to a database</font>

We'll create and connect a sqlite database using a `try-except` block. This will catch any connection errors and we can handle them in the except code block.

In [3]:
def db_connect(db_name):
    try:
        conn = sqlite3.connect(db_name)
        print("Connection established")
    except Error:
        print(Error, "Cannot create connection to database")
    return conn

Woo - with our fancy function in hand, let's try connecting to our database. We're going to be using `nobel.db`, a database on nobel laureates. Note that if our database doesn't exist, sqlite will create a new database (so be extra careful with spelling).

In [4]:
conn = db_connect("data/nobel.db")

Connection established


### <font color='#eb3483'> Running Queries </font>

To run queries (i.e. send commands) to our databse, we'll use a cursor object. Think of this as our guide to navigating the data - we give it commands, and it let's us access data.

In [ ]:
cur = conn.cursor()

Let's query the `sqlite_master` table - think of it as a master store for all the information about a database (including all it's tables). To get information about our database, we'll ask for all the information about our tables. Don't worry too much about the syntax of what we're doing yet, we'll walkthrough select statements later. 

In [ ]:
#Don't worry about what this select statement is doing quite yet, we'll circle back
query = "SELECT * FROM sqlite_master WHERE type='table';"
#This runs our query and fetches all of it's results
results = cur.execute(query).fetchall()

Let's checkout all of our table names:

In [ ]:
for table in results:
    print(table[1])

We can also get the column names of all our tables too!

In [ ]:
for table in results:
    print(table[4])

Alright great - we have a nice idea of what's in our database, let's start writing some queries to get data!

## <font color='#eb3483'> Select Statements </font>

Select statements are the bread-and-butter of data retrieval in SQL. At their core they retrieve data fields from a table.

*Side note: Placeholders for variable names are denoted in angle braces `<>`.*

**Selecting all**
```sql
SELECT *
FROM <table name>;
```
This statement returns all the entries from the given table. 
For example, if we want to inspect all of the nobel prize winners in the `laureates` table we can do:

```
SELECT * FROM laureates;
```

In [ ]:
#Let's run our select * query (we're going to start by only looking at the first 10 entries)
#What should our query be?
records = cur.execute("SELECT * FROM laureates;").fetchmany(10)
for row in records:
    print(row)

So what have we gotten back? Let's take a look at the data type.

In [ ]:
type(records[0])

A tuple! Well we're already pandas wizes - is there an easy way to get this into a dataframe? Lucky for us, pandas has some amazing functionality to integreate SQL directly into pandas dataframes. Let's import pandas and take it for a spin.

In [ ]:
#Remember it's good practice to import pandas as pd
import pandas as pd

To read a sql statement into a pandas dataframe, we can use the `read_sql` command. All we need to do is specify our sql query and the database connection to use.

In [ ]:
pd.read_sql("SELECT * FROM laureates;", conn)

Magic!

<font color='#eb3483'> **Selecting specific columns** </font>
We seldom need to select all the entries from a table for viewing purposes; it is far more common to select all when joining information onto another table, or creating an intermediate view. We're often interested in selecting a subset of a table. Column-wise this is achieved by selecting a specific column or collection of columns from the dataframe.

```sql
SELECT <column name 1>, <column name 2>
FROM <table name>;
```
The other type of subset we could select is a row-wise subset. This is typically achieved by a limit statement:
```sql
SELECT <column name 1>, <column name 2>
FROM <table name>
LIMIT 10;
```

For example, if we want to select the names of the first 10 laureates:

```sql
SELECT firstname
FROM laureates
LIMIT 10;
```

In [ ]:
pd.read_sql("SELECT firstname FROM laureates LIMIT 10;", conn)

### <font color='#eb3483'> Quick Knowledge Check </font>
1. Can you get all the data in the `prizes` table into a pandas dataframe?

## <font color='#eb3483'>Conditional Select Statements </font>
When selecting data, we often only want to select data where certain conditions are met. SQL support the following comparison operators:

1. `=` Equal to
2. `>` Greater than
3. `<` Less than
4. `>=` Greater than or equal to
5. `<=` Less than or equal to
6. `<>` Not equal to

Conditional statements make use of the comparison operators in the context of a `WHERE` clause:
```sql
SELECT <column name>
FROM <table name>
WHERE <column name> <logical operator> <value>
```
The column in the `WHERE` clause does not have to match the column in the `SELECT` statement. For instance, in the `nobel` database:

```sql
SELECT firstname, surname
FROM laureates
WHERE bornCountry = 'France';
```
Check out other logical operators like `ANY` and `BETWEEN`.

In [ ]:
pd.read_sql("SELECT firstname, surname FROM laureates WHERE bornCountry = 'France' LIMIT 5;", conn)


`WHERE` clauses can be enhanced via the use of SQL logical operators like `AND` and `OR`. Logical operators can define logic across multiple comparisons, like the following:
```SQL
SELECT *
FROM prizes
WHERE category = 'literature' AND year > 2010
LIMIT 10;
```

In [ ]:
pd.read_sql("SELECT * FROM prizes WHERE category = 'literature' AND year > 2015;", conn)

The LIKE pattern matching operator can also be used in the conditional selection of the where clause. Like is a very powerful operator that allows you to select only rows that are "like" what you specify. The percent sign "%" can be used as a wild card to match any possible character that might appear before or after the characters specified. For example:

```SQL
SELECT firstname, surname
FROM laureates
WHERE firstname LIKE 'Ma%';
```

This SQL statement will match any first names that start with 'Ma', **case insentitive**.

Or you can specify,

```SQL
SELECT firstname, surname
FROM laureates
WHERE surname LIKE '%e';
```
This statement will match any last names that end in a 'e'.



In [ ]:
pd.read_sql("SELECT * FROM laureates WHERE surname LIKE '%e' LIMIT 5;", conn)

Check out other logical operators like `ANY` and `BETWEEN`.

### <font color='#eb3483'> Quick Knowledge Check </font>
1. Can you return all the female nobel laureates who were born in France?

2. Challenge: How about all the female laureates born in Poland? Hint: If you don't have Marie Curie, you might have to think about why that might be.

## <font color='#eb3483'> Arithmetic Operators and Functions </font>
Familiar arithmetic operators and functions can also be used in the context of select statements.

For instance, the `COUNT` function is often used in the context of a `SELECT` statement to return the number of rows within a table or group:

```SQL
SELECT COUNT(*)
FROM <table name>;
```

It is often of interest to identify the number of unique values in a given column. This can be achieved by composing the `COUNT` and `DISTINCT` functions:

```SQL
SELECT COUNT(DISTINCT(<column name>))
FROM <table name>;
```

More familiar arithmetic operators (`+`, `-`, `*`, `/`, `%`) can also be applied in the context of `SELECT` statements. For instance, if we want to calculate the monthly salary from an employee database:

```SQL
SELECT salary, salary / 12 AS monthly_salary 
FROM salaries 
LIMIT 10;
```

*(notice how we can use `AS` to create **aliases**, so we can reference the monthly salary as `monthly_salary`)*

Let's compute how many different countries have nobel winners

```SQL
SELECT COUNT(DISTINCT(bornCountryCode)) 
FROM laureates 
```

In [ ]:
pd.read_sql("SELECT COUNT(DISTINCT(bornCountryCode)) AS country_count FROM laureates;", conn)

## <font color='#eb3483'> Group By Clauses </font>

Group by statements are a very useful pattern in `SQL`. They group rows of the table according to some logic, and apply aggregation functions to the selected fields. You might see this called the 'split-apply-combine' paradigm, and it's going to pop up alot. Here's an example of that paradigm on a sample customer database.

![Joins](media/groupby.png)

For instance, if we want to see how many nobel prizes have been given for each category we can do:

```SQL
SELECT category, COUNT(*) AS num_awards
FROM prizes
GROUP BY category;
```

This returns two columns. The first is the category, and the second is the number of prizes given in each category.

In [ ]:
pd.read_sql("SELECT category, COUNT(*) AS num_awards \
            FROM prizes GROUP BY category;", conn)

SQL has a few aggreagation functions we can use instead of `COUNT` including `MAX`, `MIN`, `SUM`, `AVG`. 

### <font color='#eb3483'> Quick Knowledge Check </font>
1. How many nobel laureates does every country have (count by whether or not a laureate was BORN in a country) ?

## <font color='#eb3483'>  Joining Data </font>

In our nobel database we have a table with all the specific awards (years and categories), and a table with all of our laureates, but what if we wanted to know what country the laureates who won the 2019 chemistry prize are from? To answer this question we would need information from the both tables. This is why **JOINS** are important. Joins are just a way of linking data across multiple tables. We use a column in one table, and try to match it to a column in another table.

A good way to think about joins is a Ven Diagram

![Joins](./media/sql-joins.jpg)

1. `LEFT JOIN` keeps all records matching the index field values in the left hand side dataframe, joining those from the right hand side dataframe where they exist, else filling missing values with `Null`.
2. `INNER JOIN` keeps all records matching the index field values that are present in *both* the left hand side and right hand side dataframes. It doesn't create any new `Null` values.
3. `OUTER JOIN` keeps all records that match at least one of the index field values on either the left hand side or the right hand side dataframes, and filling any missing values with `Null`.

When we join we have to chose a row to join on. For example, this could be join the prizes table and the laureaete table where the laureate_id = the laureate id. This column is called a **key**. 

Let's try joining our two tables!

```SQL
SELECT category, year, firstname, surname
FROM prizes
LEFT JOIN laureates
ON prizes.laureate_id = laureates.id;
```

In [ ]:
pd.read_sql("SELECT category, year, firstname, surname \
            FROM prizes \
            LEFT JOIN laureates \
            ON prizes.laureate_id = laureates.id;", conn)

### <font color='#eb3483'> Quick Knowledge Check </font>
1. Let's answer our original question! What country are the laureates who won the 2019 chemistry prize are from?

### <font color='#eb3483'>Commit changes and close connection</font>

When we're done with a database, it's good practice to commit all our changes, and close our connection.

In [ ]:
conn.commit()
conn.close()

Check that connection is closed

In [ ]:
cur = conn.cursor()

And with that error - we'll end our module! We've scratched the surface of SQL (we've only covered the R in CRUD), but select statements alone are really powerful tools that you'll use time and time again. Check out this [tutorial site](https://www.sqlitetutorial.net/) for more information on SQLite.

SQL is a really handy skill to have as a data scientist (more often than not you'll be pulling data from a database), so make sure to hone your SQL skills by checking out the practice problems!